<a href="https://colab.research.google.com/github/chrdrn/dbd25-nfty_scheduler/blob/main/demo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# ntfy Umfrage-Reminder - Live-Demo

Dieses Notebook ist eine Live-Demo des ntfy Umfrage-Reminder-Skripts, das in [diesem Repository](https://github.com/chrdrn/dbd25-nfty_scheduler) gehostet wird.
Dabei erfüllt es zwei zentrale Funktionen: 
- Prozess der Planung und Versendung von ntfy-Benachrichtigungen nachzuvollziehen bzw. anhand vordefinierter Paramter zu testen.
- Plattform, um die notwendingen Konfigurationen für eure eigenen ntfy-Benachrichtigungen vorzunehmen und diese zu exportieren


## Colab-Setup
Ersetze `<REPO-URL>` durch deine Repository-URL und fuehre dann diese Zelle aus.


In [1]:
!git clone https://github.com/chrdrn/dbd25-nfty_scheduler/
%cd dbd25-nfty_scheduler

Cloning into 'dbd25-nfty_scheduler'...
remote: Enumerating objects: 80, done.
remote: Counting objects: 100% (80/80), done.
remote: Compressing objects: 100% (53/53), done.
remote: Total 80 (delta 27), reused 69 (delta 22), pack-reused 0 (from 0)
Receiving objects: 100% (80/80), 32.71 KiB | 5.45 MiB/s, done.
Resolving deltas: 100% (27/27), done.
/content/dbd25-nfty_scheduler


## Aufbau der `config.py` (Defaults)
In den Beispielen am Anfang verwenden wir die Standardwerte. Die Tabelle erklaert die Argumente, aber hier musst du noch nichts anpassen.

| Argument | Typ / Beispiel | Was es bewirkt |
|---|---:|---|
| start | Datum, z.B. `2026-01-21` | Erster Tag, an dem Erinnerungen geplant werden (Beginn des Zeitraums). |
| end | Datum, z.B. `2026-01-22` | Letzter Tag, an dem Erinnerungen geplant werden (Ende des Zeitraums). |
| per_day | Ganzzahl, z.B. `5` | Wie viele Erinnerungen pro Tag gesendet werden sollen. |
| min_gap | Minuten, z.B. `60` | Minimale Pause zwischen zwei Erinnerungen am selben Tag (in Minuten). |
| mode | `interval` oder `windows` | Verteilungsmodus: `interval` = gleichmaessig im Zeitbereich; `windows` = in festgelegten Zeitfenstern. |
| interval | `HH:MM-HH:MM`, z.B. `08:00-20:00` | Nur bei `mode="interval"`: Zeitspanne, innerhalb der Erinnerungen liegen duerfen. |
| windows | Komma-getrennte Fenster, z.B. `08:00-10:00,10:00-12:00` | Nur bei `mode="windows"`: Liste von Zeitfenstern, in denen Erinnerungen platziert werden. |
| seed | Ganzzahl oder `None`, z.B. `123` | Zufalls-Seed fuer reproduzierbare Plaene. Gleiches Seed -> gleiche Planung; `None` -> echte Zufaelligkeit. |
| participant_id | Text, z.B. `gruppeA` | Kennung der Gruppe; wird z.B. im Dateinamen und in URLs verwendet. |
| out_file | Pfad, z.B. `out/gruppeA_schedule.json` | Datei, in die der erzeugte Zeitplan geschrieben wird. |


## Beispiel: Dry-Run mit Defaults
Dieser Lauf zeigt dir den echten Konsolen-Output von `plan` direkt im Notebook.


In [29]:
import subprocess

result = subprocess.run(
    ["python", "run.py", "plan", "--dry-run", "--explain"],
    check=True,
    stdout=subprocess.PIPE,
    stderr=subprocess.STDOUT,
    text=True,
)
print(result.stdout)


[explain] erlaubte Minuten pro Tag: 720 (aus 1 Fenster(n))
[explain] 2026-01-21: gewählt -> 09:58, 12:56, 14:14, 15:52, 17:30
[explain] 2026-01-22: gewählt -> 09:44, 10:50, 12:43, 14:22, 19:54
Geplante Zeitpunkte:
  ID 1: 2026-01-21 #1/5 @ 09:58 (2026-01-21T09:58)
  ID 2: 2026-01-21 #2/5 @ 12:56 (2026-01-21T12:56)
  ID 3: 2026-01-21 #3/5 @ 14:14 (2026-01-21T14:14)
  ID 4: 2026-01-21 #4/5 @ 15:52 (2026-01-21T15:52)
  ID 5: 2026-01-21 #5/5 @ 17:30 (2026-01-21T17:30)
  ID 6: 2026-01-22 #1/5 @ 09:44 (2026-01-22T09:44)
  ID 7: 2026-01-22 #2/5 @ 10:50 (2026-01-22T10:50)
  ID 8: 2026-01-22 #3/5 @ 12:43 (2026-01-22T12:43)
  ID 9: 2026-01-22 #4/5 @ 14:22 (2026-01-22T14:22)
  ID 10: 2026-01-22 #5/5 @ 19:54 (2026-01-22T19:54)

Gespeichert in: out/schedule.json

[dry-run] Kein Versand.



## Individualisierung
Ab hier passt du die Werte fuer deine Gruppe an, schaust dir eine Vorschau an und exportierst die Dateien.


In [13]:
# Individualisierte Werte (anpassen)
participant_id = "gruppeA"
start = "2026-01-21"
end = "2026-01-22"
per_day = 5
min_gap = 60
seed = 123  # set to None for random
mode = "interval"  # or "windows"
interval = "08:00-20:00"
windows = "08:00-10:00,10:00-12:00,12:00-14:00,14:00-16:00,16:00-18:00"
out_file = f"out/{participant_id}_schedule.json"


### Plan mit individuellen Werten
Dieser Chunk erzeugt deinen Schedule mit den oben gesetzten Werten.


In [14]:
import subprocess

cmd = [
    "python",
    "run.py",
    "--start",
    start,
    "--end",
    end,
    "--per-day",
    str(per_day),
    "--min-gap",
    str(min_gap),
    "--mode",
    mode,
    "--participant-id",
    participant_id,
    "--out",
    out_file,
]

if seed is not None:
    cmd += ["--seed", str(seed)]

if mode == "windows":
    cmd += ["--windows", windows]
else:
    cmd += ["--interval", interval]

cmd += [
    "plan",
    "--dry-run",
    "--explain",
]

subprocess.run(cmd, check=True)


CompletedProcess(args=['python', 'run.py', '--start', '2026-01-21', '--end', '2026-01-22', '--per-day', '5', '--min-gap', '60', '--mode', 'interval', '--participant-id', 'gruppeA', '--out', 'out/gruppeA_schedule.json', '--seed', '123', '--interval', '08:00-20:00', 'plan', '--dry-run', '--explain'], returncode=0)

### Randomisierte Vorschau (Console-Output von plan)
Dieser Chunk zeigt den Konsolen-Output von `plan --dry-run` mit deinen individuellen Werten.


In [15]:
import copy
import subprocess

preview_out_file = f"out/{participant_id}_schedule_preview.json"

plan_cmd = copy.deepcopy(cmd)
if "--out" in plan_cmd:
    out_idx = plan_cmd.index("--out")
    plan_cmd[out_idx + 1] = preview_out_file

result = subprocess.run(
    plan_cmd,
    check=True,
    stdout=subprocess.PIPE,
    stderr=subprocess.STDOUT,
    text=True,
)
print(result.stdout)


CompletedProcess(args=['python', 'run.py', '--start', '2026-01-21', '--end', '2026-01-22', '--per-day', '5', '--min-gap', '60', '--mode', 'interval', '--participant-id', 'gruppeA', '--out', 'out/gruppeA_schedule_preview.json', '--interval', '08:00-20:00', 'plan', '--dry-run', '--explain'], returncode=0)

### Schedule.json anzeigen
Dieser Chunk gibt den Inhalt der erzeugten `schedule.json` aus (basierend auf `out_file`).


In [ ]:
import json
from pathlib import Path

schedule_path = Path(out_file)
if schedule_path.exists():
    data = json.loads(schedule_path.read_text())
    print(json.dumps(data, ensure_ascii=False, indent=2))
else:
    print(f"Datei nicht gefunden: {schedule_path}")


### ntfy konfigurieren (.env)
Definiere zuerst die relevanten Werte im naechsten Chunk und fuehre dann den .env-Generator aus.


### Relevante .env-Werte
Passe diese Werte an deine Gruppe an. Diese Variablen werden im naechsten Chunk in `config/ntfy.env` geschrieben.


In [ ]:
ntfy_topic = "gruppeA"
ntfy_title = "Umfrage-Reminder {k}/{per_day}"
ntfy_message = "Bitte nimm kurz an der Umfrage teil: {url}"

survey_url_template = "https://www.soscisurvey.de/dbd25-template//?r={id}"
ntfy_server = "https://ntfy.sh"
ntfy_markdown = "1"


In [16]:
from pathlib import Path

env_path = Path("config/ntfy.env")
lines = [
    f"export NTFY_TOPIC=\"{ntfy_topic}\"",
    f"export NTFY_TITLE=\"{ntfy_title}\"",
    f"export NTFY_MESSAGE=\"{ntfy_message}\"",
]

if survey_url_template:
    lines.append(f"export SURVEY_URL_TEMPLATE=\"{survey_url_template}\"")
if ntfy_server:
    lines.append(f"export NTFY_SERVER=\"{ntfy_server}\"")
if ntfy_markdown:
    lines.append(f"export NTFY_MARKDOWN=\"{ntfy_markdown}\"")

env_text = "\n".join(lines) + "\n"
env_path.write_text(env_text)
print(f"Wrote {env_path}")


Wrote config/ntfy.env


### Optional: Testbenachrichtigung senden
Nur ausfuehren, wenn `config/ntfy.env` echte Werte enthaelt.


In [ ]:
!python run.py send 1 --explain


### Export (Pflicht)
Diese Dateien brauchst du spaeter fuer den Versand und zur Abgabe. Bitte exportiere beides am Ende.

1. `config/ntfy.env` (deine individualisierte .env)
2. `out/<id>_schedule.json` (der erzeugte Schedule; in diesem Notebook ist das `out_file`)

Der Download-Helper funktioniert nur in Colab.


In [9]:
from pathlib import Path

files_to_export = [Path(out_file), Path("config/ntfy.env")]

try:
    from google.colab import files
    print("Exportiere:")
    for p in files_to_export:
        if p.exists():
            print(f"- {p}")
            files.download(str(p))
        else:
            print(f"Missing: {p}")
except Exception:
    print("Not running in Colab.")


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

Missing: config/ntfy.env
